In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid, save_image
import os
import matplotlib.pyplot as plt

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create directory for saving images
output_dir = "/kaggle/working/generated_images"
os.makedirs(output_dir, exist_ok=True)

# Hyperparameters
latent_dim = 100       # Dimension of noise vector
img_shape = (1, 28, 28)  # Shape of MNIST images
batch_size = 64
lr = 0.0002
epochs = 50

# Transform to normalize data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize images to [-1, 1]
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define the Generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, int(torch.prod(torch.tensor(img_shape)))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

# Define the Discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(int(torch.prod(torch.tensor(img_shape))), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Initialize models and move them to GPU if available
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss and optimizer
adversarial_loss = nn.BCELoss().to(device)
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Function to generate noise on the same device
def generate_noise(batch_size, latent_dim):
    return torch.randn(batch_size, latent_dim, device=device)

# Training loop
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Move images to device
        imgs = imgs.to(device)

        # Prepare labels
        real_labels = torch.ones(imgs.size(0), 1, device=device)
        fake_labels = torch.zeros(imgs.size(0), 1, device=device)

        # Train Generator
        optimizer_G.zero_grad()
        z = generate_noise(imgs.size(0), latent_dim)
        generated_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(generated_imgs), real_labels)
        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(imgs), real_labels)
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake_labels)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

    # Log progress
    print(f"Epoch [{epoch+1}/{epochs}] - Loss D: {d_loss.item():.4f}, Loss G: {g_loss.item():.4f}")

    # Save generated images every 10 epochs
    if epoch % 10 == 0:
        with torch.no_grad():
            test_noise = generate_noise(16, latent_dim)
            test_imgs = generator(test_noise)
            grid = make_grid(test_imgs, nrow=4, normalize=True)
            save_image(grid, f"generated_images/epoch_{epoch}.png")
            print(f"Generated images saved at epoch {epoch}")

print("Training complete.")

Using device: cuda
Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:00<00:00, 14229232.43it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 470892.54it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:00<00:00, 4377517.71it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 2193245.31it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/50] - Loss D: 0.6140, Loss G: 0.4917
Generated images saved at epoch 0
Epoch [2/50] - Loss D: 0.3628, Loss G: 1.2755
Epoch [3/50] - Loss D: 0.5363, Loss G: 1.4788
Epoch [4/50] - Loss D: 0.4729, Loss G: 2.2245
Epoch [5/50] - Loss D: 0.4862, Loss G: 0.6952
Epoch [6/50] - Loss D: 0.3967, Loss G: 1.1469
Epoch [7/50] - Loss D: 0.6061, Loss G: 0.6028
Epoch [8/50] - Loss D: 0.4652, Loss G: 1.3220
Epoch [9/50] - Loss D: 0.5393, Loss G: 1.7246
Epoch [10/50] - Loss D: 0.5689, Loss G: 1.1219
Epoch [11/50] - Loss D: 0.5667, Loss G: 0.7778
Generated images saved at epoch 10
Epoch [12/50] - Loss D: 0.5913, Loss G: 1.4607
Epoch [13/50] - Loss D: 0.5403, Loss G: 0.7837
Epoch [14/50] - Loss D: 0.5712, Loss G: 0.9100
Epoch [15/50] - Loss D: 0.5225, Loss G: 0.8788
Epoch [16/50] - Loss D: 0.5730, Loss G: 1.0738
Epoch [17/50] - Loss D: 0.7263, Loss G: 1.5403
Epoch [18/50] - Loss D: 0.6133, Loss G: 1.1867
Epoch [19/50] - Los